In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import math
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [58]:
# Load data khi lấy từ lớp full connected
features_train = np.load("/kaggle/input/ge-features/features_train.npy")

In [59]:
features_train.shape

(8502, 6400)

In [60]:
# # Merge train and valid's label
train_label_df = pd.read_csv("/kaggle/input/ge-muse-split/train_label.csv")
valid_label_df = pd.read_csv("/kaggle/input/ge-muse-split/val_label.csv")
label_df = pd.concat((train_label_df, valid_label_df), axis = 0)
label_df.shape

(8502, 7)

In [61]:
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()

label_pca = label_df["Rhythm"]
label_enc = lb.fit_transform(label_pca)
label_enc

array([0, 1, 1, ..., 2, 1, 2])

In [72]:
y_test = lb.fit_transform(pd.read_csv("/kaggle/input/ge-muse-split/test_label.csv")["Rhythm"].values)
y_test.shape

(2126,)

In [ ]:
X_train = features_train
y_train = label_enc
X_test = np.load("/kaggle/input/ge-features/features_test.npy")

clf = XGBClassifier()
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.1, 0.01, 0.001],
    'subsample': [0.5, 0.7],
}

grid = GridSearchCV(clf, param_grid, refit = True, verbose = 3) 
  
grid.fit(X_train, y_train) 

In [94]:
y_pred = clf.predict(X_test)

In [95]:
y_test

array([0, 3, 0, ..., 0, 0, 2])

In [ ]:
from sklearn.metrics import classification_report

print("Kết quả không áp dụng PCA:")
print(classification_report(y_test, y_pred, target_names= lb.classes_))

result = classification_report(y_test, y_pred, target_names= lb.classes_, output_dict = True)
result_df = pd.DataFrame(result).transpose()
result_df

result_df.to_csv('./result.csv', index=True)